In [1]:
import streamlit as st
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
def load_data():
    file = 'C:/Users/HP/updated python project/env/aircrahesFullDataUpdated_2024.csv'
    df = pd.read_csv(file)

    return df

In [3]:
df = load_data()


In [4]:
df.head()

,Year,Quarter,Month,Day,Country/Region,Aircraft Manufacturer,Aircraft,Location,Operator,Ground,Fatalities (air),Aboard
0,1970,Qtr 2,April,28,NaN,de Havilland Canada,de Havilland Canada DHC 6 Twin Otter 100VH,Kainatu,Airlines Australia GuineaTrans New,0,8,11
1,1952,Qtr 2,April,30,NaN,Douglas,Douglas DC 3VT,Delhi,Airways IndiaDeccan,0,9,9
2,1973,Qtr 2,April,10,NaN,Vickers Vanguard,Vickers Vanguard 952G AXOP,Near Basel,(UK) Airlines International SwitzerlandInvicta,0,108,145
3,2000,Qtr 2,April,19,'-,Antonov,Antonov AN 8TL,Pepa -,Airlines Centrafricain,0,24,24
4,1946,Qtr 2,April,30,'-,Douglas,Douglas C 47A FZ559,Pindi-Khut -,Force Air Royal,0,12,12


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5035 entries, 0 to 5034
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Year                   5035 non-null   int64 
 1   Quarter                5035 non-null   object
 2   Month                  5035 non-null   object
 3   Day                    5035 non-null   int64 
 4   Country/Region         4805 non-null   object
 5   Aircraft Manufacturer  5035 non-null   object
 6   Aircraft               5035 non-null   object
 7   Location               5035 non-null   object
 8   Operator               4801 non-null   object
 9   Ground                 5035 non-null   int64 
 10  Fatalities (air)       5035 non-null   int64 
 11  Aboard                 5035 non-null   int64 
dtypes: int64(5), object(7)
memory usage: 472.2+ KB


In [6]:
# Step 1:  Handling Missing Values and Standardizing Entries

# Replace '-' with NaN in 'Country/Region'
df['Country/Region'] = df['Country/Region'].replace('-', pd.NA)

# Fill missing values in 'Country/Region' and 'Operator' with 'Unknown'
df['Country/Region'] = df['Country/Region'].fillna('Unknown')
df['Operator'] = df['Operator'].fillna('Unknown')

In [7]:
# Step 2: Trim and Clean Text Fields
# Columns to clean: 'Country/Region', 'Aircraft', 'Location', 'Operator'
df['Country/Region'] = df['Country/Region'].str.strip().str.title()
df['Aircraft Manufacturer'] = df['Aircraft Manufacturer'].str.strip().str.title()
df['Aircraft'] = df['Aircraft'].str.strip()
df['Location'] = df['Location'].str.strip()
df['Operator'] = df['Operator'].str.strip()

In [8]:
# Step 3: Convert 'Quarter' and 'Month' to categorical data types
df['Quarter'] = pd.Categorical(df['Quarter'], categories=['Qtr 1', 'Qtr 2', 'Qtr 3', 'Qtr 4'])
df['Month'] = pd.Categorical(df['Month'], categories=[
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'])

In [9]:
# Step 4: Handling Outliers
# Calculate the interquartile range (IQR) for 'Ground' and 'Fatalities (air)' to detect outliers
Q1_ground = df['Ground'].quantile(0.25)
Q3_ground = df['Ground'].quantile(0.75)
IQR_ground = Q3_ground - Q1_ground

Q1_fatalities = df['Fatalities (air)'].quantile(0.25)
Q3_fatalities = df['Fatalities (air)'].quantile(0.75)
IQR_fatalities = Q3_fatalities - Q1_fatalities

# Define outlier bounds
lower_bound_ground = Q1_ground - 1.5 * IQR_ground
upper_bound_ground = Q3_ground + 1.5 * IQR_ground

lower_bound_fatalities = Q1_fatalities - 1.5 * IQR_fatalities
upper_bound_fatalities = Q3_fatalities + 1.5 * IQR_fatalities

# Cap outliers in 'Ground' and 'Fatalities (air)'
df['Ground'] = df['Ground'].clip(lower=lower_bound_ground, upper=upper_bound_ground)
df['Fatalities (air)'] = df['Fatalities (air)'].clip(lower=lower_bound_fatalities, upper=upper_bound_fatalities)

In [10]:
# Step 5: Check for Duplicates
duplicate_rows = df.duplicated().sum()

# If any duplicates are found, remove them
df.drop_duplicates(inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5035 entries, 0 to 5034
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Year                   5035 non-null   int64   
 1   Quarter                5035 non-null   category
 2   Month                  5035 non-null   category
 3   Day                    5035 non-null   int64   
 4   Country/Region         5035 non-null   object  
 5   Aircraft Manufacturer  5035 non-null   object  
 6   Aircraft               5035 non-null   object  
 7   Location               5035 non-null   object  
 8   Operator               5035 non-null   object  
 9   Ground                 5035 non-null   int64   
 10  Fatalities (air)       5035 non-null   float64 
 11  Aboard                 5035 non-null   int64   
dtypes: category(2), float64(1), int64(4), object(5)
memory usage: 403.9+ KB


Research Questions Five Questions to Explore: Which aircraft manufacturers have the highest number of crashes?

What is the trend of air crashes over the years?

Which countries/regions had the highest number of fatalities?

VWhich quarters of the year experience the most air crashes?

How do air crashes relate to the number of people aboard?

Findings :

The trend of air crashes over the years shows fluctuations, with certain years experiencing higher numbers of crashes. This could be due to various factors such as changes in aviation regulations, technological advancements, or external events.

Air crashes are distributed across all quarters, but there might be slight variations in the number of crashes per quarter. This could be influenced by seasonal factors or changes in flight frequency during different times of the year.

Certain countries or regions have a higher number of air crash fatalities. This could be due to higher air traffic, less stringent safety regulations, or other regional factors.

Some aircraft manufacturers have a higher number of crashes associated with their aircraft. This could be due to the popularity of certain aircraft models, their age, or specific design issues.

Reccomendations:

Implement stricter safety regulations and regular inspections, especially in regions with higher crash rates.

Encourage airlines to upgrade or retire older aircraft models that are more prone to crashes.

Utilize data analytics to identify patterns and potential risk factors associated with air crashes.

Increase transparency in reporting air crash incidents and their causes. This can help build public trust and encourage more rigorous safety practices.

Conduct public awareness campaigns to educate passengers about safety protocols and emergency procedures.

In [12]:
df = load_data()

# Title of the app
st.title('Air Crashes Overview')

# Sidebar filters
st.sidebar.header('Filter the data')

# Filter by Year
years = st.sidebar.multiselect('Select Year(s)', options=df['Year'].unique(), default=df['Year'])
# Filter the dataframe based on the selected years
filtered_df = df[df['Year'].isin(years)]

2025-01-04 03:03:04.163 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:06.182 
  command:

    streamlit run c:\Users\HP\updated python project\env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-01-04 03:03:06.184 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:06.185 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:06.187 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:06.193 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:06.196 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:06.241 Thread 'M

In [13]:
#1 Top Aircraft Manufacturers by Number of Crashes

st.header('Top Aircraft Manufacturers by Number of Crashes')
Aircraft_Manufacturer = filtered_df['Aircraft Manufacturer'].value_counts().head(10)
plt.figure(figsize=(10,6))
plt.xlabel('Number of Crashes')
plt.ylabel('Aircraft Manufacturer')
plt.title('Top 10 Aircraft Manufacturers with the Highest Number of Crashes')
plt.gca().invert_yaxis()

st.pyplot(plt)

2025-01-04 03:03:06.310 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:06.312 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:06.433 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:07.183 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:07.184 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [14]:
#2.Air Crash Trends Over the Years
# Group by Year and count the number of crashes per year
yearly_crash_count = df.groupby('Year').size().reset_index(name='Crash Count')

st.header('Air Crash Trends Over the Years')
plt.figure(figsize=(12,6))
plt.plot(yearly_crash_count['Year'], yearly_crash_count['Crash Count'], color='blue', marker='o')
plt.xlabel('Year')
plt.ylabel('Number of Crashes')
plt.title('Trend of Air Crashes Over the Years')
plt.grid(True)

st.pyplot(plt)

2025-01-04 03:03:07.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:07.230 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:07.263 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:08.417 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:08.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [15]:
#3 Countries/Regions with the Highest Number of Air Crash Fatalities

fatalities_by_country = df.groupby('Country/Region')['Fatalities (air)'].sum().reset_index()
top_10_countries_fatalities = fatalities_by_country.sort_values(by='Fatalities (air)', ascending=False).head(10)
st.header('Countries/Regions with the Highest Number of Air Crash Fatalities')
plt.figure(figsize=(10,6))
plt.barh(top_10_countries_fatalities['Country/Region'], top_10_countries_fatalities['Fatalities (air)'], color='coral')
plt.xlabel('Number of Fatalities')
plt.ylabel('Country/Region')
plt.title('Top 10 Countries/Regions with the Highest Number of Fatalities')
plt.gca().invert_yaxis()

st.pyplot(plt)

2025-01-04 03:03:08.462 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:08.464 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:08.525 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-01-04 03:03:09.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:09.361 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [16]:
#4. Air Crashes by Quarter of the Year

crashes_by_quarter = df.groupby('Quarter').size().reset_index(name='Crash Count')
st.header('Air Crashes by Quarter of the Year')
plt.figure(figsize=(8,5))
plt.bar(crashes_by_quarter['Quarter'], crashes_by_quarter['Crash Count'], color='lightgreen')
plt.xlabel('Quarter')
plt.ylabel('Number of Crashes')
plt.title('Air Crashes by Quarter')

st.pyplot(plt)

2025-01-04 03:03:09.403 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:09.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:09.447 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:09.881 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:09.882 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [17]:
#5. Relationship Between Number of People Aboard and Air Crash Fatalities

st.header('Relationship Between Number of People Aboard and Air Crash Fatalities')
plt.figure(figsize=(8,6))
plt.scatter(df['Aboard'], df['Fatalities (air)'], color='purple', alpha=0.5)
plt.xlabel('Number of People Aboard')
plt.ylabel('Fatalities (air)')
plt.title('Relationship Between People Aboard and Air Fatalities')
plt.grid(True)

st.pyplot(plt)

2025-01-04 03:03:09.920 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:09.923 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:09.960 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:10.828 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-04 03:03:10.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()